In [1]:
%pip install youtube-transcript-api==0.5.0  # YouTube lib to get transcript
%pip install pytube==12.1.2                 # YouTube lib
%pip install transformers==4.26.1           # Used to know the token size
%pip install tiktoken==0.3.3
%pip install openai==0.27.2
%pip install langchain==0.0.134
%pip install python-dotenv==1.0.0
%pip install faiss-cpu==1.7.3

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
from dotenv import dotenv_values
import os

api_keys = dotenv_values('keys.txt')
os.environ['OPENAI_API_KEY']=api_keys['OPENAI_API_KEY']

In [3]:
from langchain.llms import OpenAI

llm = OpenAI()

In [16]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=0)

In [17]:
from langchain.document_loaders.unstructured import UnstructuredFileLoader
from langchain.document_loaders import YoutubeLoader

loader = YoutubeLoader("FtyKgybYkaE", False)
result = loader.load()

In [6]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=0)

docs = text_splitter.split_documents(result)
len(docs)

14

In [18]:
from langchain.embeddings.openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()

In [19]:
# Embed and store the texts
from langchain.vectorstores import FAISS

vectorstore = FAISS.from_documents(docs, embeddings)

In [11]:
query = "How many tickets were sold?"
search_results = vectorstore.similarity_search(query)
len(search_results)

4

In [20]:
search_results

[Document(page_content="so but if you look at it it's been three years basically and it was obviously from the beginning that people really want to get back into person events so we were assuming or we were anticipating that it would sell out fast and I will be so for devops France I think it was 11 hours Nicola 11 hours they've experience was so loud so we said like I was guessing like three days and but I said like just to play it safe we'll do two batches so we had one batch of 2000 tickets and another batch of 750 tickets and of course there's a lot of other extra tickets out just to give you the real number we are with 3200 unique developers here uh during the week so so that's the total number um and so we said the first batch we open it up we use the a very a nice registration application which I'll talk about later on and there was 1 600 tickets gone in five minutes in five minutes these were all the combi and the Deep dive sorry the combi and the conference tickets those were 

In [12]:
from langchain.chains.question_answering import load_qa_chain

chain = load_qa_chain(llm, chain_type="stuff")
chain.run(input_documents=search_results, question=query)

' 3200 tickets were sold.'

In [13]:
docs_and_scores = vectorstore.similarity_search_with_score(query)
docs_and_scores[0]

(Document(page_content="so but if you look at it it's been three years basically and it was obviously from the beginning that people really want to get back into person events so we were assuming or we were anticipating that it would sell out fast and I will be so for devops France I think it was 11 hours Nicola 11 hours they've experience was so loud so we said like I was guessing like three days and but I said like just to play it safe we'll do two batches so we had one batch of 2000 tickets and another batch of 750 tickets and of course there's a lot of other extra tickets out just to give you the real number we are with 3200 unique developers here uh during the week so so that's the total number um and so we said the first batch we open it up we use the a very a nice registration application which I'll talk about later on and there was 1 600 tickets gone in five minutes in five minutes these were all the combi and the Deep dive sorry the combi and the conference tickets those were 

In [15]:
#
# Load QnA with intermediate steps & map rerank
#
# Map-Rerank:
# This method involves running an initial prompt on each chunk of data, 
# that not only tries to complete a task but also gives a score for how certain it is in its answer. 
# The responses are then ranked according to this score, and the HIGHEST SCORE IS RETURNED.
#
from langchain.chains.qa_with_sources import load_qa_with_sources_chain

chain = load_qa_with_sources_chain(
    OpenAI(temperature=0), 
    chain_type="map_rerank", 
    metadata_keys=['source'], 
    return_intermediate_steps=True
)

result = chain({"input_documents": docs, "question": query}, return_only_outputs=True)

result["intermediate_steps"]

[{'answer': ' This document does not answer the question.', 'score': '0'},
 {'answer': ' This document does not answer the question.', 'score': '0'},
 {'answer': ' 3200 tickets were sold.', 'score': '100'},
 {'answer': ' This document does not answer the question.', 'score': '0'},
 {'answer': ' This document does not answer the question', 'score': '0'},
 {'answer': ' This document does not answer the question.', 'score': '0'},
 {'answer': ' This document does not answer the question.', 'score': '0'},
 {'answer': ' This document does not answer the question.', 'score': '0'},
 {'answer': ' This document does not answer the question.', 'score': '0'},
 {'answer': ' This document does not answer the question.', 'score': '0'},
 {'answer': ' This document does not answer the question.', 'score': '0'},
 {'answer': ' This document does not answer the question.', 'score': '0'},
 {'answer': ' This document does not answer the question.', 'score': '0'},
 {'answer': ' This document does not answer 

In [ ]:
len(docs)